In [3]:
import config
import json
import numpy as np
import pandas as pd
import requests
import warnings
import time
warnings.filterwarnings("ignore")

In [4]:
# Import clean spotify playlist & song data
data = '/Users/florencialeoni/code/omg_i_love_that_song/data/spotify_genres_popularity.csv'

In [5]:
# Create dataframe
df = pd.read_csv(data)

In [6]:
# Check dataframe first entries
df.head()

,Unnamed: 0,playlist_id,playlist_name,track_uri,track_name,artist_name,album_name,duration_ms,key,mode,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,release_date,popularity
0,0,../data/spotify1.json-1,Throwbacks,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,226864.0,4.0,0.0,...,0.904,0.813,0.006970,0.0471,-7.105,0.1210,0.810,125.461,2005-07-04,65.0
1,1,../data/spotify1.json-1,Throwbacks,6I9VzXrHxO9rA9A5euc8Ak,Toxic,Britney Spears,In The Zone,198800.0,5.0,0.0,...,0.774,0.838,0.025000,0.2420,-3.914,0.1140,0.924,143.040,2003-11-13,78.0
2,2,../data/spotify1.json-1,Throwbacks,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),235933.0,2.0,0.0,...,0.664,0.758,0.000000,0.0598,-6.583,0.2100,0.701,99.259,2003,58.0
3,3,../data/spotify1.json-1,Throwbacks,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justin Timberlake,Justified,267267.0,4.0,0.0,...,0.891,0.714,0.000234,0.0521,-6.055,0.1400,0.818,100.972,2002-11-04,72.0
4,4,../data/spotify1.json-1,Throwbacks,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Shaggy,Hot Shot,227600.0,0.0,1.0,...,0.853,0.606,0.000000,0.3130,-4.596,0.0713,0.654,94.759,2000,17.0


In [7]:
# Drop Unnamed column from dataframe
df.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [6]:
# # Create empty columns for the values that will be pulled through API
# df["release_date"] = None
# df["popularity"] = np.nan

## Drop cells that were causing errors

In [26]:
df.drop(index = 33255, inplace = True)

In [34]:
df.drop(index = 42032, inplace = True)

In [38]:
df.drop(index = 52383, inplace = True)

In [12]:
df.drop(index = 72417, inplace = True)

In [15]:
df.drop(index = 86205, inplace = True)

In [19]:
df.drop(index = 96190, inplace = True)

In [23]:
df.drop(index = 128727, inplace = True)

In [27]:
df.drop(index = 132228, inplace = True)

In [31]:
df.drop(index = 140992, inplace = True)

In [34]:
df.drop(index = 148267, inplace = True)

In [44]:
df.drop(index = 156909, inplace = True)

In [53]:
df.drop(index = 207644, inplace = True)

In [58]:
df.drop(index = 210897, inplace = True)

In [64]:
df.drop(index = 243062, inplace = True)

## API CALLS

In [65]:
# Create base URL for API calls
base_url = "https://api.spotify.com/v1/tracks/"
# Iterate row by row making calls to API with track_uri information
for index, row in df.iterrows():
    # Printing index to have a visual check on progress (know which index is being processed)
    print(f"\r{index}", end = "")
    # Check if one of the new columns is empty, if so, start making API calls from that index on
    if pd.isna(row["release_date"]):
        # Make request to API using get method
        response = requests.get(base_url + row['track_uri'],
            headers={"Authorization": "Bearer "})
        # If response is not 200 OK, print error message (to have a visual check on progress)
        while response.status_code != 200:
            print("Response error, waiting.")
            # Wait one second before making next request
            time.sleep(1)
            # Make request and check status again
            response = requests.get(base_url + row['track_uri'],
                headers={"Authorization": "Bearer "})
        
        # Use json method on the response
        track_data = response.json()
        # Fill each column of the row with the values gotten from the API response
        df.at[index, "release_date"] = track_data["album"]["release_date"]
        df.at[index, "popularity"] = track_data["popularity"]


243348Response error, waiting.
243349Response error, waiting.
243649Response error, waiting.
244167Response error, waiting.
Response error, waiting.
244453Response error, waiting.
Response error, waiting.
244551Response error, waiting.
Response error, waiting.
Response error, waiting.
Response error, waiting.
Response error, waiting.
Response error, waiting.
Response error, waiting.
244794Response error, waiting.
244972Response error, waiting.
245609Response error, waiting.
245710Response error, waiting.
Response error, waiting.
245916Response error, waiting.
Response error, waiting.
246201Response error, waiting.
246301Response error, waiting.
Response error, waiting.
246921Response error, waiting.
Response error, waiting.
247249Response error, waiting.
Response error, waiting.
247435Response error, waiting.
247535Response error, waiting.
248096Response error, waiting.
Response error, waiting.
248196Response error, waiting.
248520Response error, waiting.
249271Response error, waiting.

In [66]:
# Save the DataFrame created to a csv file
df.to_csv(r'/Users/florencialeoni/code/omg_i_love_that_song/data/spotify_genres_popularity.csv')

In [68]:
# Sanity Check
df.tail()

,playlist_id,playlist_name,track_uri,track_name,artist_name,album_name,duration_ms,key,mode,time_signature,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,release_date,popularity
266378,../data/spotify4.json-1000,other,7c1iI6dwdCpHGYT8DGEfab,Uninvited,Alanis Morissette,The Collection,276200.0,2.0,1.0,4.0,...,0.380,0.538,0.004870,0.0940,-5.493,0.0313,0.193,127.416,2005-11-11,56.0
266379,../data/spotify4.json-1000,other,4l7YIni9bAiGwWMqK5wvgV,You Learn - 2015 Remastered,Alanis Morissette,Jagged Little Pill (Remastered),239640.0,8.0,1.0,4.0,...,0.401,0.706,0.000000,0.0916,-7.554,0.0404,0.458,168.669,1995,55.0
266380,../data/spotify4.json-1000,other,2LVxvGhl2U5p2ql2ujc6vZ,Hands Clean,Alanis Morissette,Under Rug Swept,269400.0,7.0,1.0,4.0,...,0.513,0.820,0.000003,0.5040,-5.428,0.0299,0.520,99.952,2002-02-25,54.0
266381,../data/spotify4.json-1000,other,2Kgjd9G808FhUDRhS2gidG,Cecilia And The Satellite,Andrew McMahon in the Wilderness,Andrew McMahon In The Wilderness,223880.0,5.0,1.0,4.0,...,0.381,0.844,0.000000,0.3180,-7.123,0.1440,0.239,194.000,2014-10-13,1.0
266382,../data/spotify4.json-1000,other,1u0l8zWpQeMYStFkc2mLD7,Everywhere,Michelle Branch,The Spirit Room,214827.0,1.0,1.0,4.0,...,0.523,0.828,0.000255,0.1650,-4.696,0.0762,0.476,96.952,2001-07-31,66.0
